In this notebook we will train a [Logistic Regression model](https://en.wikipedia.org/wiki/Logistic_regression) to distinguish between legitimate and fraudulent transactions. 

Logistic Regression is a classic statistical technique used for binary classification. Here the binary variable we are predicting is 'legitimate' or 'not legitimate' (i.e. fraudulent).

We begin by loading in our generated data.

In [1]:
import numpy as np
import pandas as pd
df = pd.read_parquet("fraud-cleaned-sample.parquet")

We need to split our data set into two. One part will be used for training the model, and the other will be a testing set we can use to evaluate the model we train. We're dealing with time-series data, so we'll split the data set based on time.

In [2]:
first = df['timestamp'].min()
last = df['timestamp'].max()
cutoff = first + ((last - first) * 0.7)

train = df[df['timestamp'] <= cutoff].copy()
test = df[df['timestamp'] > cutoff]

We also load in the feature engineering pipeline stage which we developed in [notebook 2](02-feature-engineering.ipynb). The model takes the feature vectors as input, rather than the raw data.

In [4]:
import cloudpickle as cp
feature_pipeline = cp.load(open('feat_pipeline.pkl', 'rb'))

In [5]:
feature_pipeline

Pipeline(memory=None,
         steps=[('feature_extraction',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('interarrival_scaler',
                                                  Pipeline(memory=None,
                                                           steps=[('median_imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                

#### Dealing with Imbalanced Classes

When the training data set contains unequal representation from each of your classes we say we are dealing with 'imbalanced classes'. In our data set fewer than 2% of the samples are fraudulent, and the remaining 98% are legitimate. Thus we have imbalanced classes. 

This causes problems for a few reasons:
1. A model which classifies all transactions as 'legitimate' would be correct 98% of the time. This high accuracy can trick you into thinking that your model is working well, despite it just returning 'legitimate' for every sample it sees. 
2. Even if your model tries to learn patterns in the data, it may struggle to learn from the 'fraudulent' data since there simply isn't enough of it.


There are a few approaches we could take to tackle the problem, and today we will use two of them: 
1. We will use metrics which are more informative than simply counting how often the model makes a correct prediction. 
2. We will weight the samples by the inverse of the frequency of their label within the data set. These weights will be passed into the logistic regression model, and used to ensure that the model is penalised proportionally to this weight for making a misclassification for each class when it is training. 


In this next cell we compute these weights for each of the data labels. 

In [11]:
fraud_frequency = train[train["label"] == "fraud"]["timestamp"].count() / train["timestamp"].count()
train.loc[train["label"] == "legitimate", "weights"] = fraud_frequency
train.loc[train["label"] == "fraud", "weights"] = (1 - fraud_frequency)


We're now ready to train our Logistic Regression model. The model is trained on the feature vectors (generated using our `feature_pipeline` from the previous notebook) and we pass the class weights we computed above as a model parameter. 

In [12]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=500)

svecs = feature_pipeline.fit_transform(train)
lr.fit(svecs, train["label"], sample_weight=train["weights"])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

We need to validate our model to check how well it performs on data it wasn't trained on. We use the model we just trained to make predictions for the data in our test set, and compare those predictions to the truth. 



In [13]:
from sklearn.metrics import classification_report

predictions = lr.predict(feature_pipeline.fit_transform(test))
print(classification_report(test.label.values, predictions))


              precision    recall  f1-score   support

       fraud       0.16      0.91      0.27     14375
  legitimate       1.00      0.90      0.95    737609

    accuracy                           0.90    751984
   macro avg       0.58      0.91      0.61    751984
weighted avg       0.98      0.90      0.94    751984



The report shows that the model is performing okay, but is much better at identifying legitimate transactions than fraudulent ones. 

We can visualise the accuracy of classifications in a binary confusion matrix.

In [14]:
##from mlworkflows import plot
##df, chart = plot.binary_confusion_matrix(test["label"], predictions)
##chart

ModuleNotFoundError: No module named 'mlworkflows'

Viewing the raw counts, as well as the proportions of correctly and incorrectly classified items, emphasises that the model often misclassifies 'fraudulent' transactions as 'legitimate'. 

In [15]:
df

,timestamp,label,user_id,amount,merchant_id,trans_type,foreign,interarrival
44905449,1622396622,legitimate,8975,24.920000,12348,chip_and_pin,False,6533.0
17805936,1614921076,legitimate,3565,28.090000,12922,chip_and_pin,False,6433.0
8819718,1613602539,legitimate,1765,8.880000,15416,contactless,False,8318.0
5773423,1618232403,legitimate,1158,6.270000,14243,online,True,6674.0
8496336,1638049937,legitimate,1698,22.379999,7915,contactless,False,9609.0
...,...,...,...,...,...,...,...,...
41838350,1602623787,legitimate,8366,5.110000,3793,chip_and_pin,False,49999.0
280831,1596546511,legitimate,55,42.180000,237,swipe,False,6544.0
23291970,1589385867,legitimate,4653,7.160000,990,chip_and_pin,False,44684.0
18344979,1623603937,legitimate,3672,8.370000,13521,online,True,7802.0


We want to save the model so that we can use it outside of this notebook. 

In [16]:
cp.dump(lr, open("lr.pkl", "wb"))
